# GROMACS & CHARMM - Do a GROMACS run with a specific force field choice

We support running GROMACS with the AMBER and CHARMM force fields, specifically amber14SB-OL15 and CHARMM36-ljpme. These are the latest of each that have official ports for GROMACS and provide for protein systems.

AMBER is used by default, and both our `prepare_protein` and `gmx` modules will function in an AMBER-based MD workflow without any additional flags.

In fact, we have custom support for mapping the version 3.0 PDB residue and atom names (as per the IUPAC standards) to their AMBER-specific residue names and atom names. Thanks to this, any PDB file with these standard names, such as those from RCSB, will work seamlessly with our GROMACS module when using the AMBER force field.

When using the CHARMM force field, a compatible PDB file can be generated using `prepare_protein` by setting the `naming_scheme` configuration option to `charmm` as shown below.

## 0.0) Imports

In [ ]:
from pathlib import Path
import rush

In [ ]:
# |hide
# Users won't generally create a workspace
# We nuke to ensure run is reproducible
import os

WORK_DIR = Path.home() / "qdx" / "tutorial-gmx-resume"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

2024-03-22 16:08:41,057 - rush - INFO - Not restoring by default via default


In [ ]:
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"
client = rush.build_blocking_provider_with_functions(access_token=RUSH_TOKEN)

2024-03-22 16:08:43,749 - rush - INFO - Not restoring by default via default


In [ ]:
# |hide
# We hide this because users will generally not set a workspace, and won't restore by default
client = rush.build_blocking_provider_with_functions(
    batch_tags=["tutorial-resume-gmx"],
    workspace=WORK_DIR,
)

2024-03-22 16:08:46,611 - rush - INFO - Not restoring by default via default


## 0.1) Input Download and Selection

In [ ]:
!pdb_fetch '1B39' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'

## 1) Input Preparation
We'll make sure to pass `naming_scheme="charmm"` so that the names of the residues (i.e. amino acids) and atoms in the output will have the correct names for using with the CHARMM force field.

In [ ]:
_, prepared_protein_pdb = client.prepare_protein(
    Path.cwd() / "1B39_A_nohet.pdb",
    None,
    "charmm",
)

## 2.1) Run GROMACS (modules: gmx)
Next we will run a molecular dynamics simulation on our protein using gromacs via the `gmx` module.

We'll set "force_field" to "charmm36-ljpme", the latest official version of the CHARMM force field, in our config dictionary.

In [ ]:
_, streaming_outputs, _, *_rest = client.gmx(
    None,
    prepared_protein_pdb,
    None,
    {
        "params_overrides": {
            "nvt": {"nsteps": 2000},
            "npt": {"nsteps": 2000},
            "md": {"nsteps": 10000},
        },
        "force_field": "charmm36-ljpme",
        "num_gpus": 1,
    },
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

# Downloading Results
Let's fetch the run logs and confirm that GROMACS ran using the requested force field.

In [ ]:
streaming_outputs.download("streaming_outputs.tar.gz")

2024-03-22 16:09:01,655 - rush - INFO - Argument 96dc2630-795e-4498-a077-f28111642f57 is now ModuleInstanceStatus.RESOLVING
2024-03-22 16:09:34,280 - rush - INFO - Argument 96dc2630-795e-4498-a077-f28111642f57 is now ModuleInstanceStatus.ADMITTED
2024-03-22 16:09:40,786 - rush - INFO - Argument 96dc2630-795e-4498-a077-f28111642f57 is now ModuleInstanceStatus.DISPATCHED
2024-03-22 16:09:41,868 - rush - INFO - Argument 96dc2630-795e-4498-a077-f28111642f57 is now ModuleInstanceStatus.RUNNING
2024-03-22 16:10:40,229 - rush - INFO - Argument 96dc2630-795e-4498-a077-f28111642f57 is now ModuleInstanceStatus.AWAITING_UPLOAD


PosixPath('/home/machineer/qdx/tutorial-gmx-resume/objects/streaming_outputs.tar.gz')

In [ ]:
stdout_lines = []
for log_page in client.logs(streaming_outputs.source, kind="stderr"):
    for log in reversed(log_page):
        stdout_lines = [log] + stdout_lines
for line in stdout_lines:
    if "pdb2gmx" in line and "charmm" in line:
        print(line)

  gmx pdb2gmx -f protein_in_gmxified.pdb -o protein_in.gro -q protein_in_clean.pdb -ff charmm36-ljpme -water tip3p -noter -ss
